In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
model_path = '/content/drive/MyDrive/Master/Kaggle/Code/RestNet'
dataset_saved_path = '/content/Dataset'

In [4]:
%cd $model_path

/content/drive/MyDrive/Master/Kaggle/Code/RestNet


In [5]:
class config:
    VERSION = 3
    SUBV = "Clip_ViT_Train"

    DEVICE = 'cpu'
    SEED = 42

    # pretrained model
    RESUME = True
    RESUME_EPOCH = 0

    # backbone model
    model_type = "RestNet50"
    EFF_SIZE = 0
    EFF2_TYPE = ""
    IMAGE_SIZE = 224

    # projection layer
    N_CLASSES = 17691
    EMB_DIM = 256  
    
    # training
    TRAIN = False
    BATCH_SIZE = 256
    EPOCHS = 100
    LR = 0.00001
    save_dir = "./"
    RESIZE_SIZE = (200, 200)

# import

In [6]:
import torch, torch.nn as nn, torch.nn.functional as F
import torch.multiprocessing
import torchvision
import math
from torchvision.models import resnet50, ResNet50_Weights

# torch.multiprocessing.set_sharing_strategy('file_system')

# Model

# ResNet

In [7]:
class RestNet50(nn.Module):
  def __init__(self):
    super(RestNet50, self).__init__()
    self.model = resnet50(weights=ResNet50_Weights.DEFAULT)


    self.model.fc = torch.nn.Linear(
            self.model.fc.in_features, 1024)

    self.layer_blocks = nn.ModuleList([
            self.model.layer1, self.model.layer2, self.model.layer3,
            self.model.layer4
    ])

    self.pool_base = torch.nn.AdaptiveAvgPool2d(1)
  def forward(self, x):
    x = self.model.maxpool(
        self.model.relu(
            self.model.bn1(
              self.model.conv1(x)
            )
        )
    )
    for i, layerblock in enumerate(self.layer_blocks):
        x = layerblock(x)
        y = self.pool_base(x)
        y = y.view(y.size(0), -1)

    z = self.model.fc(y)
    return z

In [8]:
class Model(nn.Module):
  def __init__(self):
      super(Model, self).__init__()
      self.backbone_layer = RestNet50()
      self.dropout_layer = torch.nn.Dropout(p=0.2)
      self.dense_before_arcface_layer = torch.nn.Linear(
          in_features = self.backbone_layer.model.fc.out_features, 
          out_features = 256, 
          bias=True, 
      )
      self.adaptive_pooling_layer = nn.AdaptiveAvgPool1d(64)
 
  def forward(self, image):
      # image = torch.permute(image, (0,3,1,2))
      image = torchvision.transforms.functional.resize(image,size=(200, 200))
      image = image / 255.0
      image = torchvision.transforms.functional.normalize(image, 
                                          mean=[0.485, 0.456, 0.406], 
                                          std=[0.229, 0.224, 0.225])
      backbone_output = self.backbone_layer(image)
      embedded_feature = self.dropout_layer(backbone_output)
      embedded_feature = self.dense_before_arcface_layer(embedded_feature)
      embedded_feature = self.adaptive_pooling_layer(embedded_feature) 
      output = F.normalize(embedded_feature, p=2.0, dim=1, eps=1e-12, out=None)

      return output



# Make Submission File

In [9]:
output = torch.load(
    '/content/drive/MyDrive/Master/Kaggle/Code/RestNet/Training_Results/guie_CLIP_TensorFlow_train_example_with_pretrained_model_RestNet50/Model_Backups/model_guie_CLIP_TensorFlow_train_example.pt',
    map_location = torch.device('cpu'),
    )

del output['margin_layer.weight']
print(output.keys())

odict_keys(['backbone_layer.model.conv1.weight', 'backbone_layer.model.bn1.weight', 'backbone_layer.model.bn1.bias', 'backbone_layer.model.bn1.running_mean', 'backbone_layer.model.bn1.running_var', 'backbone_layer.model.bn1.num_batches_tracked', 'backbone_layer.model.layer1.0.conv1.weight', 'backbone_layer.model.layer1.0.bn1.weight', 'backbone_layer.model.layer1.0.bn1.bias', 'backbone_layer.model.layer1.0.bn1.running_mean', 'backbone_layer.model.layer1.0.bn1.running_var', 'backbone_layer.model.layer1.0.bn1.num_batches_tracked', 'backbone_layer.model.layer1.0.conv2.weight', 'backbone_layer.model.layer1.0.bn2.weight', 'backbone_layer.model.layer1.0.bn2.bias', 'backbone_layer.model.layer1.0.bn2.running_mean', 'backbone_layer.model.layer1.0.bn2.running_var', 'backbone_layer.model.layer1.0.bn2.num_batches_tracked', 'backbone_layer.model.layer1.0.conv3.weight', 'backbone_layer.model.layer1.0.bn3.weight', 'backbone_layer.model.layer1.0.bn3.bias', 'backbone_layer.model.layer1.0.bn3.running_mea

In [10]:
model = Model()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [11]:
from torchsummary import summary
summary(model, (3, config.IMAGE_SIZE, config.IMAGE_SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 100, 100]           9,408
       BatchNorm2d-2         [-1, 64, 100, 100]             128
              ReLU-3         [-1, 64, 100, 100]               0
         MaxPool2d-4           [-1, 64, 50, 50]               0
            Conv2d-5           [-1, 64, 50, 50]           4,096
            Conv2d-6           [-1, 64, 50, 50]           4,096
       BatchNorm2d-7           [-1, 64, 50, 50]             128
       BatchNorm2d-8           [-1, 64, 50, 50]             128
              ReLU-9           [-1, 64, 50, 50]               0
             ReLU-10           [-1, 64, 50, 50]               0
           Conv2d-11           [-1, 64, 50, 50]          36,864
           Conv2d-12           [-1, 64, 50, 50]          36,864
      BatchNorm2d-13           [-1, 64, 50, 50]             128
      BatchNorm2d-14           [-1, 64,

In [12]:
from PIL import Image
import torch
from torchvision import transforms

model.load_state_dict(output)
# Load image and extract its embedding.
input_image = Image.open('/content/drive/MyDrive/Master/Kaggle/Dataset/test_dataset/1_0.png').convert("RGB")
convert_to_tensor = transforms.Compose([transforms.PILToTensor()])
input_tensor = convert_to_tensor(input_image)
input_batch = input_tensor.unsqueeze(0)
print(input_batch.shape)
out = model(input_batch)[0]
# out = embedding_fn(input_batch)[0]
with torch.no_grad():
  embedding = torch.flatten(out).cpu().data.numpy()
print(embedding.shape)

torch.Size([1, 3, 373, 150])
(64,)


In [ ]:
embedding

In [ ]:
for param in model.parameters():
  print(param.data)
  break

In [ ]:
for param in model.parameters():
  print(param.data)
  break

In [13]:
model.eval()
saved_model = torch.jit.script(model)
saved_model.save('saved_model_test.pt')
from zipfile import ZipFile

with ZipFile('submission_test.zip','w') as zip:           
  zip.write('saved_model_test.pt', arcname='saved_model_test.pt')